In [7]:
import pandas as pd
import os
import re
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display
from sklearn.preprocessing import minmax_scale
import IPython.display as ipd

In [8]:
directory =' /content/drive/MyDrive/FDMDysarthria3H'

In [9]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

# **samples to show the waveforms and spectrograms:**

In [ ]:

directory = '/content/drive/MyDrive/Dysarthria_demo'
data = pd.read_csv("/content/drive/MyDrive/Dysarthria_demo/data.csv")
data['filename'] = data['filename'].apply(lambda x: os.path.join(directory,x))

data['label'] = data['is_dysarthria'].apply(lambda x: 1 if x == 'dysarthria' else 0)
data

# **Functions for showning spectrogrm and waveplots**

In [ ]:
# Function to plot waveform Samples
def show_waveplot(audio_path,label,gender):
    x , sr = librosa.load(audio_path)
    plt.figure(figsize=(10, 4))
    librosa.display.waveshow(x, sr=sr)
    plt.title(f"Waveplot: of Class: {label}, Gender: {gender}")

# Function to plot spectrogram
def show_spectrogram(audio_path,label,gender):
    x , sr = librosa.load(audio_path)
    X = librosa.stft(x)
    Xdb = librosa.amplitude_to_db(abs(X))
    plt.figure(figsize=(10,4))
    librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz',cmap='plasma')
    plt.colorbar()
    plt.title(f"Spectrogram of Class: {label}, Gender: {gender}")

# Function to plot Mel-Spectrogram
def show_melspectro(audio_path,label,gender):
    plt.figure(figsize=(10, 4))
    plt.title(f'Mel Spectro of Class: {label}, Gender: {gender}')
    x , sr = librosa.load(audio_path)
    melspectro = librosa.feature.melspectrogram(y=x, sr=sr)
    librosa.display.specshow(melspectro, sr=sr, x_axis='time',cmap='plasma')
    plt.show()

# function to plot Mfccs
def show_mfccs(audio_path,label,gender):
    plt.figure(figsize=(10, 4))
    plt.title(f'MFCC of Class: {label}, Gender: {gender}')
    x , sr = librosa.load(audio_path)
    mfccs = librosa.feature.mfcc(y=x, sr=sr)
    librosa.display.specshow(mfccs, sr=sr, x_axis='time',cmap='plasma')
    plt.show()

# **Fetching Audio Samples: Gender and Dysarthric Case:**

In [ ]:
data.sample(frac=1).reset_index(drop=True,inplace=True)
male_dysarthric = data[(data['gender']=='male') & (data['is_dysarthria']=='dysarthria')].sample(1)
female_dysarthric = data[(data['gender']=='female') & (data['is_dysarthria']=='dysarthria')].sample(1)
male_ndysarthric = data[(data['gender']=='male') & (data['is_dysarthria']=='non_dysarthria')].sample(1)
female_ndysarthric = data[(data['gender']=='female') & (data['is_dysarthria']=='non_dysarthria')].sample(1)

## **Male Dysarthric voice:**

In [ ]:
ipd.Audio(male_dysarthric.iloc[0].filename)

In [ ]:
show_waveplot(male_dysarthric.iloc[0].filename, male_dysarthric.iloc[0].is_dysarthria, male_dysarthric.iloc[0].gender)

# **Female dysarthric voice:**

In [ ]:
ipd.Audio(female_dysarthric.iloc[0].filename)

In [ ]:
show_waveplot(female_dysarthric.iloc[0].filename, female_dysarthric.iloc[0].is_dysarthria, female_dysarthric.iloc[0].gender)

# **Spectrogram of dysarthic voice:**

In [ ]:
show_spectrogram(male_dysarthric.iloc[0].filename, male_dysarthric.iloc[0].is_dysarthria, male_dysarthric.iloc[0].gender)
show_spectrogram(female_dysarthric.iloc[0].filename, female_dysarthric.iloc[0].is_dysarthria, female_dysarthric.iloc[0].gender)



```
`# This is formatted as code`
```

## **MFCC Feature Extraction for Random forest and GMM:**
## We performed this experiment on entire torgo data set

In [ ]:
import pandas as pd
import librosa
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, recall_score
import numpy as np
import torch


In [ ]:
# def getting_file_name(file_name):
#   file_split=file_name.split('_')
#   folder_name='not'
#   if "C" in file_split[0]:
#     folder_name='not'
#   else:
#     folder_name='Yes'
#   return folder_name


In [ ]:
def getting_folder_name(folder_path):
  file_split=folder_path.split('/')
  ff=file_split[-1].split('_')
  if ff[0]=='dysarthria':
    folder_name='dysarthria'
    return folder_name
  else:
    folder_name='non_dysarthria'
    return folder_name

In [ ]:
mfcc_final=[]
root_folder='/content/drive/MyDrive/Dysarthria_demo/torgo_data'
# folder='/content/drive/MyDrive/Dysartrhia_dataset/torgo_data'
label=[]
lab_val=0
for folder in os.listdir(root_folder):
  curr_folder=os.path.join(root_folder,folder)
  print("curr",curr_folder)
  # print(len(os.listdir(curr_folder)))
  for file in os.listdir(curr_folder):
    # print("file",file,os.path.join(curr_folder,file))
    # print(len(os.listdir(curr_folder)))


    try:
      x,sr=librosa.load(os.path.join(curr_folder,file))
      mfccs = librosa.feature.mfcc(y=x, sr=sr)
      row, col = mfccs.shape
      mfcc_t=mfccs.T
      print(np.shape(mfcc_t))
      row_t,col_t=mfcc_t.shape
      # print("tras",row_t,col_t)
      # print(mfccs[0].shape)

      folder_name=getting_folder_name(curr_folder)
      # print("folder name",folder_name)

      if folder_name=='dysarthria':
        for k in range(row_t):
          label.append(1)
          mfcc_final.append(mfcc_t[k])
      else:
        for k in range(row_t):
          label.append(0)
          mfcc_final.append(mfcc_t[k])
    except:
      print("probelmatic file",file)



In [ ]:
print(len(mfcc_final),len(label))
X=np.array(mfcc_final)
Y=np.array(label)

# **Random forest classifer:**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.30)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf =  RandomForestClassifier()
clf.fit(xtrain,ytrain)


In [ ]:
ypred=clf.predict(xtest)
score=accuracy_score(ytest, ypred)
print("accuracy",score)

#**Section to run if you have already a trained model**

In [ ]:
# # dont run this cell
# import pickle

# filename = "/content/drive/MyDrive/Dysarthria_demo/random_forest.pickle"

# # save model
# pickle.dump(clf, open(filename, "wb"))

# # load model
# loaded_model = pickle.load(open(filename, "rb"))


# **Gaussian Mixture Model:**

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
n_components = 16
gmm = GaussianMixture(n_components=n_components, random_state=42)

# Fit the model to the training data
gmm.fit(xtrain)

# Predict labels on the test set
y_pred = gmm.predict(xtest)

# Evaluate accuracy
accuracy = accuracy_score(ytest, y_pred)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

#**Section to run if you have already a trained model**

In [ ]:
# import pickle

# filename = "/content/drive/MyDrive/Dysarthria_demo/gmm.pickle"

# # save model
# pickle.dump(gmm, open(filename, "wb"))

# # load model
# loaded_model = pickle.load(open(filename, "rb"))

# **MFCC feature extraction for SVM and DNN :**  
## We performe this experiment on a subset of torgo datset

In [ ]:
mfcc_final=[]
root_folder='/content/drive/MyDrive/Dysarthria_demo/torgo_data_sample_1'
# folder='/content/drive/MyDrive/Dysartrhia_dataset/torgo_data'
label=[]
lab_val=0
for folder in os.listdir(root_folder):
  curr_folder=os.path.join(root_folder,folder)
  print("curr",curr_folder)
  # print(len(os.listdir(curr_folder)))
  for file in os.listdir(curr_folder):
    # print("file",file,os.path.join(curr_folder,file))
    # print(len(os.listdir(curr_folder)))


    try:
      x,sr=librosa.load(os.path.join(curr_folder,file))
      mfccs = librosa.feature.mfcc(y=x, sr=sr)
      row, col = mfccs.shape
      mfcc_t=mfccs.T
      print(np.shape(mfcc_t))
      row_t,col_t=mfcc_t.shape
      # print("tras",row_t,col_t)
      # print(mfccs[0].shape)

      folder_name=getting_folder_name(curr_folder)
      print("folder name",folder_name)

      if folder_name=='dysarthria':
        for k in range(row_t):
          label.append(1)
          mfcc_final.append(mfcc_t[k])
      else:
        for k in range(row_t):
          label.append(0)
          mfcc_final.append(mfcc_t[k])
    except:
      print("probelmatic file",file)

In [ ]:
print(len(mfcc_final),len(label))
x=np.array(mfcc_final)
y=np.array(label)

# **SVM classifier:**

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.15)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
svm_clf =  SVC(kernel='rbf')
svm_clf.fit(x_train,y_train)


In [ ]:
y_pred=svm_clf.predict(x_test)
score=accuracy_score(y_test, y_pred)
print("accuracy",score)

#**Loading and saving the model:**

In [ ]:
import pickle

filename = "/content/drive/MyDrive/Dysarthria_demo/svm_clf.pickle"

# save model
pickle.dump(svm_clf, open(filename, "wb"))

# load model
loaded_model = pickle.load(open(filename, "rb"))

# **Deep Neural network:**

In [ ]:
import torch.nn as nn
import torch.optim as optim

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
X_train_tensor = torch.from_numpy(x_train).to(device)
y_train_tensor = torch.from_numpy(y_train).to(device)
X_test_tensor = torch.from_numpy(x_test).to(device)
y_test_tensor = torch.from_numpy(y_test).to(device)

In [ ]:
# a,b=X_tensor.shape
# c=y_tensor.shape
# print(a,b,c)


In [ ]:
class DNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        # print(x.shape)
        x = self.fc2(x)
        # print(x.shape)
        return x
print(X_train_tensor.shape[0],y_train_tensor.shape[0])
model = DNN(X_train_tensor.shape[1], 128, y_train_tensor.shape[0]).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(10):
    for i in range(X_train_tensor.shape[0]):
        optimizer.zero_grad()
        # print(X_tensor[i],X_tensor[i].shape)
        outputs = model(X_train_tensor[i])
        loss = criterion(outputs, y_train_tensor[i])

        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')



In [ ]:
# Set the model to evaluation mode
model.eval()

with torch.no_grad():
    new_data_tensor = X_test_tensor[10]
    output = model(new_data_tensor)
    _, predicted_class = torch.max(output, 0)

print(f'Predicted Class: {predicted_class.item()}')

In [ ]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for i in range(X_test_tensor.shape[0]):
        outputs = model(X_test_tensor[i])
        _, predicted = torch.max(outputs.data, 0)
        total += 1
        correct += (predicted == y_test_tensor[i]).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')